In [1]:
import argparse
import sys, os
sys.path.append(os.path.join(sys.path[0], '/home/ibespalov/unsupervised_pattern_segmentation/'))
sys.path.append(os.path.join(sys.path[0], '/home/ibespalov/unsupervised_pattern_segmentation/gans_pytorch/'))
sys.path.append(os.path.join(sys.path[0], '/home/ibespalov/unsupervised_pattern_segmentation/gans_pytorch/stylegan2'))
sys.path.append(os.path.join(sys.path[0], '/home/ibespalov/unsupervised_pattern_segmentation/gans_pytorch/gan/'))
from gan.gan_model import cont_style_munit_enc
from models.munit.enc_dec import MunitEncoder
from parameters.dataset import DatasetParameters
from parameters.deformation import DeformationParameters
from parameters.gan import GanParameters, MunitParameters

import albumentations
import torch
from albumentations.pytorch.transforms import ToTensor as AlbToTensor
from dataset.cardio_dataset import ImageMeasureDataset
from torch.utils import data
from transforms_utils.transforms import MaskToMeasure
from dataset.d300w import ThreeHundredW
from dataset.probmeasure import ProbabilityMeasureFabric, ProbabilityMeasure
import numpy as np

In [9]:
device = torch.device("cuda:3" if torch.cuda.is_available() else "cpu")
print(device)

cuda:3


In [10]:
from torch import nn

In [11]:
linear_layer = nn.Linear(10,2).cuda()

In [12]:
opt = torch.optim.Adam(linear_layer.parameters(), lr=0.03)

In [13]:
linear_parallel_model = nn.DataParallel(linear_layer, [0, 1, 2])

In [14]:
input_tensor = torch.randn((8,10)).cuda()

In [16]:
linear_layer.weight

Parameter containing:
tensor([[ 0.0507,  0.0180,  0.2655, -0.2965,  0.2900,  0.0445, -0.1661, -0.0893,
         -0.1243,  0.1989],
        [-0.0234,  0.1321, -0.0669,  0.1198, -0.1414,  0.2823,  0.0564, -0.2042,
          0.1247,  0.1711]], device='cuda:0', requires_grad=True)

In [17]:
final_tensor = linear_parallel_model(input_tensor)

In [18]:
final_tensor.sum().backward()

In [19]:
opt.step()

In [20]:
linear_layer.weight

Parameter containing:
tensor([[ 0.0207, -0.0120,  0.2355, -0.2665,  0.3200,  0.0745, -0.1361, -0.1193,
         -0.1543,  0.2289],
        [-0.0534,  0.1021, -0.0969,  0.1498, -0.1114,  0.3123,  0.0864, -0.2342,
          0.0947,  0.2011]], device='cuda:0', requires_grad=True)

In [23]:
linear_layer.weight[0][0] = 0.5

In [25]:
linear_parallel_model.module.weight

Parameter containing:
tensor([[ 0.5000, -0.0120,  0.2355, -0.2665,  0.3200,  0.0745, -0.1361, -0.1193,
         -0.1543,  0.2289],
        [-0.0534,  0.1021, -0.0969,  0.1498, -0.1114,  0.3123,  0.0864, -0.2342,
          0.0947,  0.2011]], device='cuda:0', grad_fn=<CopySlices>)